# Data Preperation

## Imports

In [20]:
# data processing
import numpy as np
import pandas as pd

# plotting
import seaborn as sns
import matplotlib.pyplot as plt

## Preperation & Cleaning

In [2]:
def change_rel_time(group):
    group["rel_time"] -= min(group["rel_time"])
    
    group["rel_time_hour"] =  group["rel_time"] / 60
    group["rel_time_hour"] = group["rel_time_hour"].apply(np.floor)
    group["rel_time_hour"] = group["rel_time_hour"].astype(int).round(0)
    return group

In [3]:
path = "../master-thesis/data/datenbank_werte_df.parquet"

# Load the dataset
datenbank_werte_df = pd.read_parquet(path)
print("Dataset loaded...")

# Only select rows that are from the UKE Database
datenbank_werte_df = datenbank_werte_df[datenbank_werte_df["DB"] == "UKE"]
print("Removed eICU rows...")

# We can combine the Maßnahme with the ID column, becasue there is no extra information
datenbank_werte_df["Maßnahme"] += "_" + datenbank_werte_df["ID"]
datenbank_werte_df = datenbank_werte_df.drop(columns="ID")
print("Combined ID and Maßnahme and drop ID...")

# Droppinbg Maßname_norm since it's irrelevant when working only with UKE data
datenbank_werte_df = datenbank_werte_df.drop(columns="Maßnahme_norm")
print("Dropped Maßnahme_norm...")

# Change the dataframe from a long format to a wide format
datenbank_werte_df = datenbank_werte_df.pivot_table(index=["rel_time", "Pat_ID", "DB"], columns="Maßnahme", values="Wert").reset_index()
print("Pivoted Table...")

# change the column names to better more uniform column names
datenbank_werte_df.columns = [x.lower().replace(" ", "_").replace("-", "_") for x in datenbank_werte_df.columns]
print("Cleaned column names")

# Shift the rel_time so that it begins at 0
datenbank_werte_df = datenbank_werte_df.groupby("pat_id").apply(lambda group: change_rel_time(group))
print("shifted relative time to start at 0...")

# Merge down columns to a 60 minute format
datenbank_werte_df.set_index(['pat_id', 'rel_time'])    
datenbank_werte_df = datenbank_werte_df.groupby(["pat_id","rel_time_hour"]).mean().reset_index()
datenbank_werte_df["rel_time"] = datenbank_werte_df["rel_time_hour"] * 60
datenbank_werte_df = datenbank_werte_df.drop(columns="rel_time_hour")

datenbank_werte_df.head()

Load dataset
Removed eICU rows...
Combined ID and Maßnahme and drop ID...
Dropped Maßnahme_norm...
Pivoted Table...
Cleaned column names
shifted relative time to start at 0...


,pat_id,rel_time,af_vital,alt_labor,ast_labor,albumin_labor,alk_labor,alpha_blocker_med,barbiturat_med,basophile_labor,...,diast_vital,ft3_labor,ft4_labor,mittl_nbd_vital,mittl_vital,ph_bga,pankreasspez_labor,so2_bga,syst_nbd_vital,syst_vital
0,123456798,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,123456798,3120,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,123456798,13200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,123456798,38040,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,123456798,43680,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [53]:
datenbank_werte_df.to_parquet("../master-thesis/data/database_values_wide.parquet")